In [ ]:

# === Step 2: Merge into a Single data.h5 File ===
import h5py
import numpy as np

with h5py.File('/train_catvnoncat.h5', 'r') as train_file, \
     h5py.File('/test_catvnoncat.h5', 'r') as test_file, \
     h5py.File('data.h5', 'w') as data_file:

    data_file.create_dataset('train_set_x', data=np.array(train_file['train_set_x']))
    data_file.create_dataset('train_set_y', data=np.array(train_file['train_set_y']))
    data_file.create_dataset('test_set_x', data=np.array(test_file['test_set_x']))
    data_file.create_dataset('test_set_y', data=np.array(test_file['test_set_y']))
    data_file.create_dataset('list_classes', data=np.array(test_file['list_classes']))

# === Step 3: Logistic Regression with Neural Network Mindset ===
import matplotlib.pyplot as plt

def load_dataset():
    dataset = h5py.File("data.h5", "r")
    train_set_x_orig = np.array(dataset["train_set_x"][:])
    train_set_y_orig = np.array(dataset["train_set_y"][:])
    test_set_x_orig = np.array(dataset["test_set_x"][:])
    test_set_y_orig = np.array(dataset["test_set_y"][:])
    classes = np.array(dataset["list_classes"][:])
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def initialize_with_zeros(dim):
    w = np.zeros((dim, 1))
    b = 0
    return w, b

def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)
    cost = -1/m * np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    dw = 1/m * np.dot(X, (A - Y).T)
    db = 1/m * np.sum(A - Y)
    cost = np.squeeze(cost)
    grads = {"dw": dw, "db": db}
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost=False):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost)
            if print_cost:
                print(f"Cost after iteration {i}: {cost}")
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    return params, grads, costs

def predict(w, b, X):
    A = sigmoid(np.dot(w.T, X) + b)
    return (A > 0.5).astype(int)

def model(X_train, Y_train, X_test, Y_test, num_iterations=2000, learning_rate=0.005, print_cost=False):
    w, b = initialize_with_zeros(X_train.shape[0])
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)
    print(f"Train Accuracy: {100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100:.2f}%")
    print(f"Test Accuracy: {100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100:.2f}%")
    return {
        "costs": costs,
        "Y_prediction_test": Y_prediction_test,
        "Y_prediction_train": Y_prediction_train,
        "w": w,
        "b": b,
        "learning_rate": learning_rate,
        "num_iterations": num_iterations
    }

# === Step 4: Prepare the Data and Train ===
train_x_orig, train_y, test_x_orig, test_y, classes = load_dataset()

train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

train_x = train_x_flatten / 255.
test_x = test_x_flatten / 255.

d = model(train_x, train_y, test_x, test_y, num_iterations=2000, learning_rate=0.005, print_cost=True)

# === Step 5: Plot the Cost Curve ===
plt.plot(d["costs"])
plt.ylabel("cost")
plt.xlabel("iterations (per hundreds)")
plt.title(f"Learning rate = {d['learning_rate']}")
plt.show()
